In [1]:
pip install pymavlink

                                              0.0/6.2 MB ? eta -:--:--
                                              0.0/6.2 MB ? eta -:--:--
                                              0.0/6.2 MB ? eta -:--:--
                                              0.0/6.2 MB ? eta -:--:--
                                              0.1/6.2 MB 419.2 kB/s eta 0:00:15
                                              0.1/6.2 MB 438.1 kB/s eta 0:00:15
                                              0.1/6.2 MB 386.4 kB/s eta 0:00:16
                                              0.1/6.2 MB 449.7 kB/s eta 0:00:14
     -                                        0.2/6.2 MB 541.9 kB/s eta 0:00:12
     -                                        0.2/6.2 MB 541.9 kB/s eta 0:00:12
     -                                        0.2/6.2 MB 541.9 kB/s eta 0:00:12
     --                                       0.3/6.2 MB 655.4 kB/s eta 0:00:10
     --                                       0.3/6.2 MB 655.4 kB/s eta 0:00

In [2]:
from pymavlink.dialects.v20 import ardupilotmega as mavlink


In [6]:
from pymavlink.dialects.v10 import ardupilotmega as mavlink

def make_mavlink_v1(src_sys=42, src_comp=200):
    m = mavlink.MAVLink(None)
    m.srcSystem = src_sys
    m.srcComponent = src_comp
    if hasattr(m, "mavlink20"):
        m.mavlink20(False)
    return m

def make_parser_v1():
    p = mavlink.MAVLink(None)
    if hasattr(p, "mavlink20"):
        p.mavlink20(False)
    if hasattr(p, "robust_parsing"):
        p.robust_parsing = True
    return p

def pack_heartbeat(m):
    kw = dict(
        type=mavlink.MAV_TYPE_GENERIC,
        autopilot=mavlink.MAV_AUTOPILOT_INVALID,
        base_mode=0,
        custom_mode=0,
        system_status=0
    )
    try:
        kw["mavlink_version"] = 1
        msg = mavlink.MAVLink_heartbeat_message(**kw)
    except TypeError:
        kw.pop("mavlink_version", None)
        msg = mavlink.MAVLink_heartbeat_message(**kw)
    return msg.pack(m)

def parse_bytes_v1(parser, data_bytes):
    out = []
    for b in data_bytes:
        msg = parser.parse_char(bytes([b]))
        if msg is not None:
            out.append(msg)
    return out

def explain_v1_frame(frame):
    stx = frame[0]; n = frame[1]; seq = frame[2]; sysid = frame[3]; compid = frame[4]; msgid = frame[5]
    payload = frame[6:6+n]; crc = frame[6+n:8+n]
    print("Raw (hex):", " ".join(f"{b:02X}" for b in frame))
    print("STX:", hex(stx), "LEN:", n, "SEQ:", seq, "SYSID:", sysid, "COMPID:", compid, "MSGID:", msgid)
    print("PAYLOAD:", " ".join(f"{b:02X}" for b in payload))
    print("CRC:", " ".join(f"{b:02X}" for b in crc))

tx = make_mavlink_v1(42, 200)
rx = make_parser_v1()

hb_frame = pack_heartbeat(tx)
explain_v1_frame(hb_frame)

decoded = parse_bytes_v1(rx, hb_frame)
for m in decoded:
    print("Decoded:", m.get_type(), m.to_dict())

assert hb_frame[0] == 0xFE, f"Expected v1 start 0xFE, got {hb_frame[0]:#04x}"
assert decoded and decoded[0].get_type() == "HEARTBEAT"
print("✅ HEARTBEAT v1 build & decode OK")


Raw (hex): FE 09 00 2A C8 00 00 00 00 00 00 08 00 00 01 9D EB
STX: 0xfe LEN: 9 SEQ: 0 SYSID: 42 COMPID: 200 MSGID: 0
PAYLOAD: 00 00 00 00 00 08 00 00 01
CRC: 9D EB
Decoded: HEARTBEAT {'mavpackettype': 'HEARTBEAT', 'type': 0, 'autopilot': 8, 'base_mode': 0, 'custom_mode': 0, 'system_status': 0, 'mavlink_version': 1}
✅ HEARTBEAT v1 build & decode OK
